In [16]:
import pandas as pd 
import numpy as np
import os

print("this is working")

tubefolder = os.getcwd() + "\TubeFile2020\\"
tubefile = tubefolder + "ski1_tubelist.csv"

print(tubefile)


data = pd.read_csv(tubefile)

this is working
C:\Users\f002tj9\Documents\Research\kj\TubeFile2020\ski1_tubelist.csv


In [22]:
#Saving all of the stuff into np arrays (vectors)

type(data)
X = np.asarray(data['x'])
Y = np.asarray(data['y'])
A = np.asarray(data['a'])
B = np.asarray(data['b'])
ROT = np.asarray(data['rot'])
F = np.asarray(data['f'])
VX = np.asarray(data['Vx'])
VY = np.asarray(data['Vy'])
TNUM = np.asarray(data['tubenum'])
print(type(X))
print(ROT[0:10])
print(len(VY))

<class 'numpy.ndarray'>
[ 1.56433482e+00  1.51407252e+00 -1.47737167e+00 -1.29707856e+00
 -1.30493964e+00 -1.29085234e+00  5.86669755e-02  5.86669755e-02
 -8.95106453e-02 -2.26221010e-12]
7810
